In [1]:
import utils
from utils import load_data, num_to_char ,load_video
from modelutil import load_model
import joblib
import tensorflow as tf


In [2]:
import dash
from dash import dcc
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
from dash import dcc, html, dash_table
from jupyter_dash import JupyterDash
import pandas as pd
import dash_bootstrap_components as dbc


In [3]:

import base64
import datetime
import io
import dash_daq as daq

In [4]:



app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUMEN])



drop_video=dcc.Upload(contents="",filename="",
                                        id='upload-image',
                                        children=html.Div([
                                            'Drag and Drop or ',
                                            html.A('Select Files')
                                                        ]),
                                                            style={
                                                                'width': '50%',
                                                                'height': '60px',
                                                                'lineHeight': '60px',
                                                                'borderWidth': '1px',
                                                                'borderStyle': 'dashed',
                                                                'borderRadius': '5px',
                                                                'textAlign': 'center',
                                                                'margin': '10px'
                                                                },        # Allow multiple files to be uploaded
                                                                multiple=False)


video = dbc.Card(
    dbc.CardBody([
        
    ],id="video_uploaded"),
)





In [5]:
# Define the callback for when a video is uploaded
@app.callback(Output('video_uploaded', 'children'),
              [Input('upload-image', 'contents')],
              [State('upload-image', 'filename')])
def update_video(contents, filename):
    if contents is not None:
        children = parse_contents(contents, filename)
        return children
    return ""
    
    

def parse_contents(contents, filename):
    return  [ html.Video(
            id="video-player",
            controls=True,
            src = contents,
            style={"width": "100%"}
        ), html.H2(str(filename),className="card-title",style={'fontSize': 25, 'font-family':'Courier New, monospace','align':'center'}),
        html.Button("read the lips",id="predict_button",className="btn btn-warning",style={"margin":"auto"}),
        html.Div(html.Label('Predicted answer.', id="show_answer",style={'color': '#068DA9', 'borderWidth': '1px',
                                                'borderStyle': 'dashed',
                                                'borderRadius': '5px',
                                                'textAlign': 'center','fontSize': 25, 'width': '90%',
                                                'font-family':'Courier New, monospace','margin':'15px'}),id="model_output")]


In [6]:
#return generated text


@app.callback(Output('show_answer', 'children'),
              [Input('predict_button', 'n_clicks')],
              [State('upload-image', 'contents')])
def update_output(n_clicks, contents):
    if n_clicks is not None:
        # Decode the base64-encoded string to bytes
        video_bytes = base64.b64decode(str(contents).split(',')[1])

        # Write the bytes to a file
        with open('video.mp4', 'wb') as f:
            f.write(video_bytes)
          
        text = open_model("video.mp4")

    return text
    

#callback to display generated text has the model
def open_model(path):
    
    video=load_video(path)
    model = load_model()
#     model = joblib.load("model_.joblib")
#     model = load_model('my_model.h5')
    
    output= model.predict(video.numpy().reshape(1 , 75 , 46 , 140 , 1 ))
    
    decoded = tf.keras.backend.ctc_decode(output, input_length=[75], greedy=True)[0][0].numpy()
    
    prediction = [tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in decoded]
    
    predictipn =prediction[0].numpy().decode('utf-8')
    
    return prediction[0].numpy().decode('utf-8')

### Main Layout

In [11]:
app.layout = dbc.Container([
    dbc.Row([dbc.Col([
        drop_video,html.P("Please make sure the video is mp4",style={"margin":"center"})
    ],width=6,align="center")],className="six columns",justify="center"),\

    dbc.Row([
        dbc.Col(video, width=6),
        ],className="six columns",justify="center"),

], fluid=True)



In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False,port=8050)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Jun/2023 20:01:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2023 20:01:08] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2023 20:01:08] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2023 20:01:08] "GET /_dash-component-suites/dash/dcc/async-upload.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2023 20:01:08] "POST /_dash-update-component HTTP/1.1" 200 -
[2023-06-05 20:01:08,845] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Yasmeen Nour\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Yasmeen Nour\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Yasmeen Nour\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, ex

1/1 [==============================] - 6s 6s/step


127.0.0.1 - - [05/Jun/2023 20:01:56] "POST /_dash-update-component HTTP/1.1" 200 -
